In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, roc_auc_score,roc_curve

breast_cancer=pd.read_csv('breast-cancer-wisconsin.data')

breast_cancer.columns=['id_number','clumb_thickness','unif_cell_size','unif_cell_shape'
                      ,'marg_adesion','single_epith_cell_size','bare_nuciei','bland_chromatin'
                      ,'normal_nuclioli','mitoses','class']

breast_cancer['bare_nuciei']=breast_cancer['bare_nuciei'].replace('?',np.nan)
breast_cancer['bare_nuciei']=breast_cancer['bare_nuciei'].fillna(breast_cancer['bare_nuciei'].value_counts().index[0])

breast_cancer['bare_nuciei']=breast_cancer['bare_nuciei'].replace('?',np.nan)
breast_cancer['bare_nuciei'].fillna(breast_cancer['bare_nuciei'].value_counts().index[0])
breast_cancer.columns=['id_number','clumb_thickness','unif_cell_size','unif_cell_shape'
                      ,'marg_adesion','single_epith_cell_size','bare_nuciei','bland_chromatin'
                      ,'normal_nuclioli','mitoses','class']
breast_cancer['cancer_ind']=0

breast_cancer.loc[breast_cancer['class']==4,'cancer_ind']=1

x=breast_cancer.drop(['id_number','class','cancer_ind'],axis=1)
y=breast_cancer['cancer_ind']

x_train, x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=1)

scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)  #스케일 한다고 항상 성능이 좋아지는 것은 아님 

knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train_scaled,y_train)
y_pred=knn.predict(x_test_scaled)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, roc_auc_score,roc_curve

breast_cancer=pd.read_csv('breast-cancer-wisconsin.data')
breast_cancer

,1000025,5,1,1.1,1.2,2,1.3,3,1.4,1.5,2.1
0,1002945,5,4,4,5,7,10,3,2,1,2
1,1015425,3,1,1,1,2,2,3,1,1,2
2,1016277,6,8,8,1,3,4,3,7,1,2
3,1017023,4,1,1,3,2,1,3,1,1,2
4,1017122,8,10,10,8,7,10,9,7,1,4
...,...,...,...,...,...,...,...,...,...,...,...
693,776715,3,1,1,1,3,2,1,1,1,2
694,841769,2,1,1,1,2,1,1,1,1,2
695,888820,5,10,10,3,7,3,8,10,2,4
696,897471,4,8,6,4,3,4,10,6,1,4


In [118]:
accuracy_score(y_test,y_pred)

0.9619047619047619

In [119]:
confusion_matrix(y_test,y_pred)

array([[140,   2],
       [  6,  62]], dtype=int64)

In [120]:
roc_auc_score(y_test,y_pred)

0.9488400994200497

### 그리드 서치 적용

In [121]:
from sklearn.model_selection import GridSearchCV
grid_search=GridSearchCV(knn, {'n_neighbors':[1,2,3,4,5]},n_jobs=-1, cv=7, scoring='roc_auc')
grid_search.fit(x_train_scaled,y_train)    #scoring에 따라 정확도 달라짐 

GridSearchCV(cv=7, estimator=KNeighborsClassifier(n_neighbors=1), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5]}, scoring='roc_auc')

In [122]:
grid_search.best_params_

{'n_neighbors': 5}

In [123]:
knn_best=grid_search.best_estimator_

In [124]:
y_pred=knn_best.predict(x_test_scaled)

y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0], dtype=int64)

In [125]:
accuracy_score(y_test,y_pred)

0.9761904761904762

In [126]:
confusion_matrix(y_test,y_pred)

array([[141,   1],
       [  4,  64]], dtype=int64)

In [127]:
roc_auc_score(y_test,y_pred)

0.9670671085335543